In [1]:
#modules needed 
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier
import category_encoders
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

Loading Data

In [2]:
cat_pred=pd.read_csv(os.path.join(os.getcwd(),'cat_train_prob.csv'))
for_pred=pd.read_csv(os.path.join(os.getcwd(),'forest_train_prob.csv'))
for_smal_pred=pd.read_csv(os.path.join(os.getcwd(),'forest_smal_train_prob.csv'))
NN_pred=pd.read_csv(os.path.join(os.getcwd(),'NN_train_prob.csv'))
H2O_1_pred=pd.read_csv(os.path.join(os.getcwd(),'h20_1_train_prob.csv'))
H2O_2_pred=pd.read_csv(os.path.join(os.getcwd(),'h20_2_train_prob.csv'))
H2O_3_pred=pd.read_csv(os.path.join(os.getcwd(),'h20_3_train_prob.csv'))

train=pd.read_csv(os.path.join(os.getcwd(),'train.csv.zip'))

### Metrics

In [3]:
#class_weights={1: 0.37062, 2: 0.49657,3:0.05947,4:0.018,5:0.018,6:0.018,7:0.018}
class_weights={1: 0.37053, 2: 0.49657,3:0.05947,4:0.00106,5:0.01287,6:0.02698,7:0.03238}
list_weight=compute_class_weight(class_weights,np.unique(train['Cover_Type']),train['Cover_Type'])

In [4]:
#columns
target_col='Cover_Type'
soil_type=[x for x in train.columns if 'Soil' in x]
wild_type=[x for x in train.columns if 'Wilderness' in x]
vert_dist=['Vertical_Distance_To_Hydrology']
hor_dist=['Horizontal_Distance_To_Hydrology','Horizontal_Distance_To_Roadways',  'Horizontal_Distance_To_Fire_Points']
hill_feat=['Hillshade_9am', 'Hillshade_Noon','Hillshade_3pm']
other_feat=['Elevation', 'Aspect', 'Slope']

In [5]:
def bal_acc(y_true,y_pred,weights=None):
    #weights should be provided as list of weights for each class to sum up to 1
    temp1=np.unique(y_true,return_counts=True)[1]
    temp2=confusion_matrix(y_true,y_pred)
    acc_bal=np.sum(np.diag(temp2)*(temp1/np.sum(temp1))/temp1)
    if len(weights)==0:
        acc_weig=acc_bal
    elif len(weights)!=len(temp1):
        print('provide weight for each class')
    else:
        acc_weig=np.sum(np.diag(temp2)*weights/temp1)
    return (acc_weig,acc_bal)

### Check individual 

In [6]:
print('cat_acc ',bal_acc(train[target_col].values,(np.argmax(cat_pred.iloc[:,1:].values,1)+1),weights=list_weight))
print('for_acc ',bal_acc(train[target_col].values,(np.argmax(for_pred.iloc[:,1:].values,1)+1),weights=list_weight))
print('NN_acc ',bal_acc(train[target_col].values,(np.argmax(NN_pred.iloc[:,1:].values,1)+1),weights=list_weight))

cat_acc  (0.772510162037037, 0.8607142857142857)
for_acc  (0.786329462962963, 0.8853174603174603)
NN_acc  (0.7926306018518519, 0.8728174603174602)


In [7]:
print('H2O_1_acc ',bal_acc(train[target_col].values,(np.argmax(H2O_1_pred.iloc[:,2:].values,1)+1),weights=list_weight))
print('H2O_2_acc ',bal_acc(train[target_col].values,(np.argmax(H2O_2_pred.iloc[:,2:].values,1)+1),weights=list_weight))
print('H2O_3_acc ',bal_acc(train[target_col].values,(np.argmax(H2O_3_pred.iloc[:,2:].values,1)+1),weights=list_weight))

H2O_1_acc  (0.7762913379629628, 0.8805555555555555)
H2O_2_acc  (0.7755452962962962, 0.879100529100529)
H2O_3_acc  (0.7745515370370372, 0.8794973544973544)


In [8]:
temp=cat_pred.iloc[:,1:].values+for_pred.iloc[:,1:].values+NN_pred.iloc[:,1:].values+H2O_1_pred.iloc[:,2:].values+H2O_2_pred.iloc[:,2:].values

In [9]:
print('mean_acc ',bal_acc(train[target_col].values,(np.argmax(temp,1)+1),weights=list_weight))

mean_acc  (0.801738449074074, 0.89265873015873)


In [10]:
H2O_bagged=np.mean([H2O_1_pred.iloc[:,2:].values,H2O_2_pred.iloc[:,2:].values,H2O_3_pred.iloc[:,2:].values],axis=0)

In [11]:
H2O_bagged.shape

(15120, 7)

In [12]:
X_data=np.hstack([cat_pred.iloc[:,1:].values,for_pred.iloc[:,1:].values,for_smal_pred.iloc[:,1:].values,NN_pred.iloc[:,1:].values,H2O_bagged])

### Tuning RF

In [13]:
#first I split the data
X_train, X_val, y_train, y_val = train_test_split(X_data, train[target_col], stratify=train[target_col],  test_size=0.2,random_state=42)

In [14]:
#one more time but this version will be needed for cross validation
from sklearn.metrics import make_scorer
def bal_acc2(y_true,y_pred):
    #weights should be provided as list of weights for each class to sum up to 1
    temp1=np.unique(y_true,return_counts=True)[1]
    temp2=confusion_matrix(y_true,y_pred)
    acc_bal=np.sum(np.diag(temp2)*(temp1/np.sum(temp1))/temp1)
    weights=[0.37053, 0.49657, 0.05947, 0.00106, 0.01287, 0.02698, 0.03238]
    acc_weig=np.sum(np.diag(temp2)*weights/temp1)
    return (acc_weig)

my_score = make_scorer(bal_acc2)

In [15]:
def Bayes_RandomTrees2(n_estimators, max_depth,max_features):
    
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    
    assert type(n_estimators) == int
    assert type(max_depth) == int

    #etc =RandomForestClassifier(n_estimators = n_estimators,max_features = max_features,bootstrap = False,max_depth=max_depth,verbose=0,n_jobs=-1,random_state=42)
    etc =RandomForestClassifier(n_estimators = n_estimators,max_features = max_features,bootstrap = False,max_depth=max_depth,verbose=0,n_jobs=-1,class_weight=class_weights,random_state=42)
    etc.fit(X_train, y_train)
    
    score = bal_acc2(y_val, etc.predict(X_val))
    
    return score

In [16]:
params = {'n_estimators' : (80, 400),
          'max_depth' : (5, 20),
          'max_features' : (.1,0.8)} 

RandomTreeBO = BayesianOptimization(Bayes_RandomTrees2, params, random_state = 42)

print(RandomTreeBO.space.keys)


['max_depth', 'max_features', 'n_estimators']


In [17]:
init_points = 50
n_iter = 150

In [18]:
RandomTreeBO.maximize(init_points = init_points,  n_iter = n_iter,   acq = 'ucb',    xi = 0.0,        alpha = 1e-6)

|   iter    |  target   | max_depth | max_fe... | n_esti... |
-------------------------------------------------------------
|  1        |  0.8049   |  10.62    |  0.7655   |  314.2    |
|  2        |  0.8119   |  13.98    |  0.2092   |  129.9    |
|  3        |  0.8232   |  5.871    |  0.7063   |  272.4    |
|  4        |  0.8111   |  15.62    |  0.1144   |  390.4    |
|  5        |  0.8117   |  17.49    |  0.2486   |  138.2    |
|  6        |  0.8194   |  7.751    |  0.313    |  247.9    |
|  7        |  0.8174   |  11.48    |  0.3039   |  275.8    |
|  8        |  0.8188   |  7.092    |  0.3045   |  197.2    |
|  9        |  0.8058   |  11.84    |  0.6496   |  143.9    |
|  10       |  0.807    |  12.71    |  0.5147   |  94.86    |
|  11       |  0.8175   |  14.11    |  0.2194   |  100.8    |
|  12       |  0.7937   |  19.23    |  0.7759   |  338.7    |
|  13       |  0.8224   |  9.569    |  0.1684   |  299.0    |
|  14       |  0.8202   |  11.6     |  0.1854   |  238.5    |
|  15   

|  80       |  0.8199   |  11.11    |  0.1028   |  190.3    |
|  81       |  0.8187   |  5.074    |  0.1068   |  106.0    |
|  82       |  0.815    |  5.109    |  0.1157   |  189.4    |
|  83       |  0.8054   |  20.0     |  0.1103   |  225.7    |
|  84       |  0.8193   |  5.026    |  0.1556   |  376.5    |
|  85       |  0.8158   |  5.349    |  0.1083   |  239.3    |
|  86       |  0.8228   |  11.96    |  0.1093   |  265.2    |
|  87       |  0.8177   |  12.81    |  0.1011   |  289.5    |
|  88       |  0.8171   |  12.0     |  0.1021   |  361.6    |
|  89       |  0.8032   |  10.43    |  0.7856   |  352.0    |
|  90       |  0.8011   |  19.9     |  0.1612   |  372.9    |
|  91       |  0.8175   |  5.073    |  0.109    |  302.8    |
|  92       |  0.8157   |  5.0      |  0.1      |  126.8    |
|  93       |  0.8237   |  9.871    |  0.1076   |  104.8    |
|  94       |  0.8052   |  20.0     |  0.1175   |  129.7    |
|  95       |  0.8027   |  19.98    |  0.1043   |  254.8    |
|  96   

KeyboardInterrupt: 

In [ ]:
print(RandomTreeBO.max)

### Sample fitting

In [55]:
#forest=RandomForestClassifier(n_estimators = 300,max_features = 0.9,bootstrap = False,max_depth=8,class_weight=class_weights)
forest=RandomForestClassifier(n_estimators = 400,max_features = 'auto',bootstrap = False,max_depth=8,class_weight=class_weights)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False,
                       class_weight={1: 0.37053, 2: 0.49657, 3: 0.05947,
                                     4: 0.00106, 5: 0.01287, 6: 0.02698,
                                     7: 0.03238},
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=350, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [56]:
#the accuracy of validation
print("special acc is: ",bal_acc(y_train,forest.predict(X_train),weights=list_weight))

special acc is:  (0.9157714120370369, 0.8596230158730158)


In [57]:
predict=forest.predict(X_val)

In [58]:
#the accuracy of validation
print("special acc is: ",bal_acc(y_val,predict,weights=list_weight))

special acc is:  (0.8249881944444446, 0.8293650793650793)


Results are: 

350 auto 8 (0.8249881944444446, 0.8293650793650793)
300 auto 5 (0.8192396296296297, 0.6610449735449736)
300 auto 6 (0.823106875, 0.6917989417989419)
300 auto 9 (0.8208907175925927, 0.8432539682539683)
300 auto 8 (0.8239894444444446, 0.828042328042328)
300 0.1 7 (0.8158878935185185, 0.7681878306878306)
300 auto 7  (0.8218199074074075, 0.7582671957671956)
200, auto, 10 (0.8265940277777779, 0.8531746031746031)
300, 0.9, depth=8 all h2O here special acc is:  (0.8186674305555555, 0.689484126984127)

In [59]:
CrossValidationscores = cross_val_score(forest, X_data, train[target_col], cv=5, scoring = my_score)

In [60]:
np.mean(CrossValidationscores)

0.8356124953703704

CV of 3+1+1 H2o model 

200,auto,10 is 0.8304926666666667
300 auto 7 is 0.836686449074074  (that was the one who gave the best result
300 0.1 7 0.8345641157407409
300 auto 8 0.8354721203703704
300 auto 9 0.8314878796296297
300 auto 6 0.8376620972222224
300 auto 5 0.8338968287037039
350 auto 8 0.8356124953703704

CV of 5+1 model 0.8202221712962963


forest_full=RandomForestClassifier(n_estimators = 300,max_features = 'auto',bootstrap = False,max_depth=7,class_weight=class_weights)
forest_full.fit(X_data_3, train[target_col])

Now adding the Gaussian Mixture

In [220]:
test=pd.read_csv(os.path.join(os.getcwd(),'test.csv.zip'))

In [93]:
from sklearn.mixture import GaussianMixture
features_cluster=soil_type+wild_type+hor_dist+['Elevation']
gmix = GaussianMixture(n_components=11)
gmix.fit(test[features_cluster])

temp_train_cluster = gmix.predict(train[features_cluster])
temp_test_cluster = gmix.predict(test[features_cluster])

In [221]:
#X_data_3=np.hstack([cat_pred.iloc[:,1:].values,for_pred.iloc[:,1:].values,NN_pred.iloc[:,1:].values])
#X_data_3=np.hstack([X_data,np.expand_dims(temp_train_cluster,-1)])
X_data_3=X_data

In [222]:
forest_3=RandomForestClassifier(n_estimators = 300,max_features = 'auto',bootstrap = False,max_depth=7,class_weight=class_weights)

In [223]:
CrossValidationscores_2 = cross_val_score(forest_3, X_data_3, train_new[target_col], cv=5, scoring = my_score)

In [224]:
np.mean(CrossValidationscores_2)

0.828467412037037

### Making final prediction with bagging

In [61]:
cat_pred_test=pd.read_csv(os.path.join(os.getcwd(),'cat_test_prob.csv'))
for_pred_test=pd.read_csv(os.path.join(os.getcwd(),'forest_test_prob.csv'))
for_smal_pred_test=pd.read_csv(os.path.join(os.getcwd(),'forest_smal_test_prob.csv'))
NN_pred_test=pd.read_csv(os.path.join(os.getcwd(),'NN_test_prob.csv'))
H2O_1_test_pred=pd.read_csv(os.path.join(os.getcwd(),'h20_1_test_prob.csv'))
H2O_2_test_pred=pd.read_csv(os.path.join(os.getcwd(),'h20_2_test_prob.csv'))
H2O_3_test_pred=pd.read_csv(os.path.join(os.getcwd(),'h20_3_test_prob.csv'))
test=pd.read_csv(os.path.join(os.getcwd(),'test.csv.zip'))

In [62]:
X_test=np.hstack([cat_pred_test.iloc[:,1:].values,for_pred_test.iloc[:,1:].values,for_smal_pred_test.iloc[:,1:].values,NN_pred_test.iloc[:,1:].values,H2O_1_test_pred.iloc[:,2:].values,H2O_2_test_pred.iloc[:,2:].values,H2O_3_test_pred.iloc[:,2:].values])
H2O_bagged_test=np.mean([H2O_1_test_pred.iloc[:,2:].values,H2O_2_test_pred.iloc[:,2:].values,H2O_3_test_pred.iloc[:,2:].values],axis=0)
X_test_2=np.hstack([cat_pred_test.iloc[:,1:].values,for_pred_test.iloc[:,1:].values,for_smal_pred_test.iloc[:,1:].values,NN_pred_test.iloc[:,1:].values,H2O_bagged_test])

#X_test=np.hstack([X_test,np.expand_dims(temp_test_cluster,-1)])


In [63]:
def train_forest(data_train,y_train,data_val,bagg=5):
    temp_result=[]
    for i in range(bagg):
        forest_model=RandomForestClassifier(n_estimators = 350,max_features = 'auto',bootstrap = False,max_depth=8,class_weight=class_weights,verbose=0,random_state=i*10)
        #forest_model=RandomForestClassifier(n_estimators = 300,max_features = 0.9,bootstrap = False,max_depth=8,class_weight=class_weights,verbose=0,random_state=i*10)
        #forest_model=RandomForestClassifier(n_estimators = 100,max_features = 'auto',bootstrap = False,max_depth=87,verbose=0,class_weight=class_weights,random_state=i*10)
        forest_model.fit(data_train, y_train)
        temp_result.append(forest_model.predict_proba(data_val))
        print(i,' fit finished')
    return np.mean(temp_result,axis=0)

In [64]:
forest_full=train_forest(X_data,train[target_col],X_test_2,bagg=5)

0  fit finished
1  fit finished
2  fit finished
3  fit finished
4  fit finished


In [65]:
pred_test=np.argmax(forest_full,axis=1)+1

In [66]:
sub=pd.DataFrame(test.Id.values,columns=['ID'])
sub['Cover_Type']=pred_test
sub.to_csv('merged_forest_tuned_v3.csv',index=False)

In [67]:
sub_prob=pd.DataFrame(test.Id.values,columns=['ID'])
sub_prob=pd.concat([sub_prob,pd.DataFrame(forest_full)])
sub_prob.to_csv('merged_forest_tuned_v3_proba.csv',index=False)

Result 
v3 0.84437 RandomForestClassifier(n_estimators = 350,max_features = 'auto',bootstrap = False,max_depth=8,class_weight=class_weights,verbose=0,random_state=i*10) with bagging 5 mean H2O + 4 models

v2 0.84161 RandomForestClassifier(n_estimators = 200,max_features = 'auto',bootstrap = False,max_depth=10,class_weight=class_weights,verbose=0,random_state=i*10) with 5 bagging and mean of H2O + 4 more models

v1 0.83316   RandomForestClassifier(n_estimators = 300,max_features = 0.9,bootstrap = False,max_depth=8,class_weight=class_weights,verbose=0,random_state=i*10) with 10 bagging and 6 data

### Next Step is knn smoothing

In this part I will use the fact that data is more or less sorted by labels; So I will use knn co compare results with neighbors and change the probabilities of labels considering neighbors.

In [68]:
from sklearn.neighbors import KNeighborsClassifier

In [69]:
metric_knn='minkowski'
n_neighbors_knn=4
weights_knn='uniform'
#weights_knn='distance'
algorithm_knn='auto'
#algorithm_knn="brute"

In [70]:
neigh = KNeighborsClassifier(n_neighbors=n_neighbors_knn, n_jobs=-1, metric=metric_knn,algorithm=algorithm_knn)

In [71]:
neigh.fit(np.expand_dims(test.Id.values,axis=-1), pred_test) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
                     weights='uniform')

In [72]:
prob=neigh.predict_proba(np.expand_dims(test.Id.values,axis=-1))

In [73]:
knn_pred_prob=prob*1+forest_full
knn_pred=np.argmax(knn_pred_prob,axis=1)+1

In [74]:
np.sum(np.abs(pred_test-knn_pred)>0)

5719

In [75]:
sub_knn=pd.DataFrame(test.Id.values,columns=['ID'])
sub_knn['Cover_Type']=knn_pred
sub_knn.to_csv('merged_forest_knn_1_v2.csv',index=False)

0.75 gave growth from 0.84161 to 0.84434
1.5 gave 0.84462 - slightly better

With forest of 350, auto and 8 from 0.84434 to 0.84698

With 2 neightbors

In [76]:
metric_knn='minkowski'
#n_neighbors_knn=2
weights_knn='uniform'
#weights_knn='distance'
algorithm_knn='auto'
#algorithm_knn="brute"

In [87]:
neigh_2 = KNeighborsClassifier(n_neighbors=6, n_jobs=-1, metric=metric_knn,algorithm=algorithm_knn)
neigh_2.fit(np.expand_dims(test.Id.values,axis=-1), pred_test) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=6, p=2,
                     weights='uniform')

In [88]:
prob_knn_2=neigh_2.predict_proba(np.expand_dims(test.Id.values,axis=-1))

In [89]:
knn_pred_prob_2=prob_knn_2*1+forest_full
knn_pred_2=np.argmax(knn_pred_prob_2,axis=1)+1

In [90]:
np.sum(np.abs(pred_test-knn_pred_2)>0)

9477

In [91]:
sub_knn=pd.DataFrame(test.Id.values,columns=['ID'])
sub_knn['Cover_Type']=knn_pred_2
sub_knn.to_csv('merged_forest_knn6_1_v2.csv',index=False)

This exact code gave 0.84900!!!! so 6 knn smoothing works!!!